# Práctica  2. Deep Learning para clasificación
---

**Sistemas Inteligentes para la Gestión en la Empresa**

Máster Profesional en Ingeniería Informática. Universidad de Granada.

Curso académico 2022-2023

---
**Autores:**
- Ramón García Verjaga (rgarver@correo.ugr.es)
- José Alberto Gómez García (modej@correo.ugr.es)

---

## Introducción

En los últimos años, el campo de la visión por computador ha experimentado un rápido avance gracias a los avances en el aprendizaje automático y, en particular, en las redes neuronales profundas. En esta segunda práctica de la asignatura "Sistemas Inteligentes para la Gestión en la Empresa" haremos uso de modelos de clasificación basados en redes neuronales profundas para abordar la clasificación de aves, en función de su especie, a partir de imágenes de las mismas.

La clasificación de aves es una tarea desafiante debido a la gran variabilidad y similitudes entre las especies. El conjunto de datos CUB-200-2011, abreviatura de Caltech-UCSD Birds-200-2011, es una referencia comúnmente utilizada en la comunidad de visión por computador para abordar esta tarea y probar el desempeño de distintos modelos de clasificación. El conjunto original, disponible en la web [Caltech Library](https://data.caltech.edu/records/65de6-vp158) contiene más de 11.000 imágenes de 200 especies de aves diferentes, con una amplia diversidad de apariencias y posturas. Además del conjunto original, usaremos una versión reducida de este conjunto de datos, con algo más de 1.000 imágenes de 20 especies de aves distintas.

Para desarrollar esta práctica haremos uso del lenguaje de programación Python, así como del framework para el desarrollo de redes neuronales profundas PyTorch. Adicionalmente, necesitaremos de otras librerías, como Numpy, Matplotlib y Scikit-Learn para almacenar datos, generar gráficas y realizar ciertos cálculos. Por tanto, para la ejecución de este cuaderno es necesario disponer de estos paquetes software. Si se ejecuta este cuaderno en Google Colab todos los requisitos están satisfechos; de ejecutarse en una máquina local deben ser instalados haciendo uso del comando `pip install -r requirements.txt`. Se recomienda hacer uso de un entorno Linux, pues ocasiona menos problemas para instalar y usar PyTorch con soporte con GPU, imprescindible si queremos entrenar redes neuronales en un tiempo asumible.

Para que el cuaderno pueda ejecutarse sin ningún problema, el directorio que lo contiene (resultado de descomprimir el fichero entregado) deberá tener la siguiente estructura:

- dataset/
    - data additional/
    - data x20/
    - data x200/
    - results/
        - model_X_SOME_CHARACTERISTICS.pt
        - model_Y_OTHER_CHARACTERISTICS.pt
    - best_training_logs/
        - model_X_METHOD_X.txt
        - model_Y_METHOD_Y.txt 
- img_for_docs/
- CODE_AND_REPORT.ipynb
- requirements.txt

Nótese que la estructura dentro de la carpeta `dataset` es la proporcionada en el fichero comprimido que se nos proporcionó con el conjunto de datos (con alguna consideración que discutiremos posteriormente). La carpeta `img_for_docs` simplemente contiene imágenes que se mostrarán durante este cuaderno. Dado el gran tamaño del conjunto de datos, no se adjuntan las carpetas `data x20` ni `data x200`, pero sí que se adjunta la carpeta `data additional`. El directorio `results` se generará automáticamente de no existir, pero en principio contendrá un subdirectorio con los logs impresos en la terminal durante la ejecución de los mejores entrenamientos, cuyos detalles no se han abordado en este cuaderno. Los ficheros con los modelos deben ser descargados, como se explicará más adelante. 

## Configuración del entorno de trabajo

En esta sección nos centraremos fundamentalmente en preparar el entorno de ejecución del que haremos uso a lo largo de la práctica.

En primer lugar definiremos 2 variables para determinar el contexto general de ejecución del programa.

- `USING_COLAB` especifica si nos encontramos haciendo uso del servicio en la nube de Google Colab (con el que podemos acceder gratuitamente a GPUs) o si ejecutamos el software en local. 
Este servicio ha sido usado para poder desarrollar el cuaderno de forma conjunta, y se llegó a utilizar para realizar algunas ejecuciones. Dado que los tiempos de ejecución eran muy elevados y en ocasiones se producían bloqueos del programa y desconexiones, finalmente se realizaron las ejecuciones en el ordenador de José Alberto. Para contextualizar los tiempos de ejecución que se mostrarán posteriormente, este ordenador dispone de un procesador Intel Core i7 8700K, 32 GB de memoria RAM DDR4, un SSD Samsung 970 Evo Plus de 1 TB y una tarjeta gráfica NVIDIA GTX 1080 (8 GB de VRAM).
- `EXECUTING_FOR_FIRST_TIME` es un flag que especifica si se deben ejecutar operaciones que solo deben ejecutarse una única vez, como comprobar la integridad de los datos, generar algunas gráficas en el marco del análisis exploratorio y demás. Si durante la corrección de esta práctica se ejecuta el cuaderno, se recomienda mantener su valor en `False`.

In [1]:
USING_COLAB = False
EXECUTING_FOR_FIRST_TIME = False

Posteriormente, cargaremos todas las librerías necesarias. 

Como se mencionó en la introducción, necesitaremos de PyTorch, Numpy, Matplotlib y Scikit-Learn. Adicionalmente, se usan algunas librerías pre-instaladas de Python, como la necesaria para usar funcionalidades dependientes del sistema operativo, fundamentalmente para la gestión de ficheros y rutas (`os`), la librería de expresiones regulares (`re`), la librería para manejar aleatoriedad (`ramdom`) o la que habilita la paralelización de procesos en CPU (`multiprocessing`). De hacer uso de Google Colab, los datos deben ser cargados de Google Drive, por lo que neecistamos la librería que habilita la conexión entre ambos servicios.

In [2]:
import os, re, random
import torch
import torchvision as tv
import torch.nn.functional as F
import torchsummary
import numpy as np
import multiprocessing as mp
import matplotlib as mpl
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import sklearn.model_selection as skms
import sklearn.metrics as skmetrics
from typing import Tuple
if USING_COLAB:
  from google.colab import drive
  %pip install torchsummary

Posteriormente, nos situamos en la carpeta donde se encuentra el dataset con `os.chdir`. Es importante mencionar que deberemos encontrarnos en la carpeta `practica2`, padre de este cuaderno y demás elementos de la entrega, para que se puedan usar las rutas definidas en este cuaderno sin necesidad de modificar nada. De usar Google Colab, deberá modificarse la ruta donde se encuentra el datset, de acuerdo con la organización de sus ficheros en este servicio. En cualquier caso, se asume que el conjunto de datos se encuentra descomprimido.

Posteriormente, definimos dos constantes para distinguir entre el dataset reducido y el completo, generamos las rutas del sistema de ficheros a la carpeta con cada conjunto de datos y creamos la carpeta donde guardaremos los resultados (de no existir).

In [3]:
if USING_COLAB:
  # Mount Google Drive and move to dataset directory (it is already uploaded and unzipped)
  drive.mount('/content/drive/')
  os.chdir('/content/drive/MyDrive/Práctica 2 - Deep Learning para clasificación/dataset')
else: # Local, move to folder of dataset asuming we are place on root directory for this practice (images already unzipped)
  os.chdir(os.path.join(os.getcwd(), 'dataset'))
  # chdir is done, so it execution fails, kernel needs to be restarted to avoid errors (or insert a os.chdir(../) to go back to root directory)))

DATASET_20 = 'data x20'
DATASET_200 = 'data x200'
DATASET_20_PATH = os.path.join(os.getcwd(), DATASET_20) # No / at the end
DATASET_200_PATH = os.path.join(os.getcwd(), DATASET_200)

RANDOM_SEED = 42  # Keep consistency in random generators
OUT_DIR = 'results'
try:  # No overwrite already existing results
  os.makedirs(OUT_DIR, exist_ok=False)
except FileExistsError:
  print('Results directory already exists, skipping creation.')

Results directory already exists, skipping creation.


Dado que la ejecución de los diferentes entrenamientos de los modelos de clasificación podían tomar un tiempo considerable, se proporcionan una serie de variables que actuan como flags. Estas permiten "activar" y "desactivar" el entrenamiento de los diferentes modelos, así como su evaluación sobre el conjunto de test. También podemos controlar si queremos generar las gráficas con la evolución de las funciones de pérdida y la precisión.

Dado que no se espera que se ejecute este cuaderno se entrega con todos los flags desactivados. De querer ejecutar algo, recuerde colocar los ficheros con los pesos pre-entrenados en el directorio `dataset/results`. Los ficheros con los pesos pre-entrenados se pueden descargar desde [este enlace a Zenodo](https://zenodo.org/record/7995859) (asegúrese de seleccionar la versión 2). Dado que no hemos utilizado previamente Zenodo, se proporciona [este enlace alternativo a MEGA](https://mega.nz/file/3joRRDzI#BLq3jt8LStjD6NeFaisGv8QKB-n4yIlLY4gdrTV4kHk), donde los ficheros permanecerán disponibles hasta la publicación de las actas de la asignatura por si hubiera algún tipo de problema. De no poder acceder de ninguna de las dos maneras, por favor contactar con la mayor celeridad posible a los correos especificados en el inicio de este documento.

In [4]:
TRAIN_BASELINE_MODELS = False
TRAIN_PRETRAINED_MODELS = False
TRAIN_EFFICIENTNET_MODELS = False
TRAIN_MOBILENET_MODELS = False
TRAIN_SMALL_MODELS = False
TRAIN_BIG_MODELS = False
TEST_SMALL_MODELS = False
TEST_BIG_MODELS = False
GENERATE_GRAPHS = False

Finalmente, comprobamos si disponemos de GPU o si deberemos recurrir de la CPU del equipo. En la medida de lo posible, se recomienda encarecidamente hacer uso de GPU para poder realizar la ejecución en un tiempo asumible. De hacerse uso de Google Colab, recuerde que debe habilitar la GPU manualmente, y que la duración máxima de las ejecuciones dependerá de su plan de pago y del estado de los recursos de Google. En el plan gratuito, la ejecución finaliza a las 12 horas, tiempo que en principio debería ser más que suficiente.

In [5]:
# Setting up torch's device. (In Collab GPU has to be enable manually, and only for limited time)
if torch.cuda.is_available():
  DEVICE = torch.device("cuda")
  print(f'There are {torch.cuda.device_count()} GPU(s) available.')
  print(f'Using GPU: {torch.cuda.get_device_name(0)}')
else:
  DEVICE = torch.device("cpu")
  print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
Using GPU: NVIDIA GeForce GTX 1080


## Análisis exploratorio


Como en cualquier proyecto de ciencia de datos, en primer lugar deberemos analizar el conjunto de datos con el que vamos a trabajar, de manera que tengamos un mejor entendimento del problema y podamos dilucidar que técnicas son las adecuadas para pre-procesar y manejar el conjunto de datos.

Las tres funciones que se definen a continuación se encargan de obtener las rutas a todos los ficheros correspondientes a imágenes dentro de un determinado directorio, comprobar si una determinada imagen está corrupta y hacer la comprobación anterior para todo un directorio (eliminando aquellas que efectivamente estén corruptas). 

Para calcular si la imagen está corrupta se hace uso de la desviación estándar del valor de los píxeles; de ser esta cercana a 0 todos los píxeles tendrán un color igual o muy similar, síntoma habitual de que la imagen está corrupta y debe ser eliminada. 

En función del número de imágenes a procesar este cálculo puede tardar algunos minutos, por lo que se paraleliza el trabajo en tantos hilos de CPU como se pueda.

In [6]:
def get_filepaths(path_to_data: str, fileformat: str='.jpg') -> list:
  """
  Returns paths to files of the specified format
  """
  filepaths = list()
  for root, _, filenames in os.walk(path_to_data):
    for fn in filenames:
      if fn.lower().endswith(fileformat):
        filepaths.append(os.path.join(root,fn))

  return filepaths

def check_image_is_corruputed(path_to_img: str) -> Tuple[bool, str]:
  """
  Return if the image is corrupted and the path to it
  """
  std = np.std(mpimg.imread(path_to_img))
  img_ok = not np.isclose(std, 0.0)
  return img_ok, path_to_img

In [7]:
def check_and_delete_corrupt_images(path_to_dataset: str, dataset: str) -> None:
  """
  Deletes the corrupt images given a dataset path and its name. 
  This takes quite some time.
  """
  print("Going to use {} cores".format(mp.cpu_count()))
  
  data_file_paths =  get_filepaths(path_to_dataset)
  print(len(data_file_paths), "images in", dataset)

  imgs_corrupted = list()
  with mp.Pool(processes=mp.cpu_count()) as pool:
    for img_ok, fn in pool.imap_unordered(check_image_is_corruputed, data_file_paths):
      if not img_ok:
          imgs_corrupted.append(fn)

  print('Corrupted images:', len(imgs_corrupted))
  for fn in imgs_corrupted:
      os.remove(fn)

La comprobación y eliminación de imágenes corruptas sólo debe realizarse en la primera carga del conjunto de datos, por lo que se encuentra bajo el flag `EXECUTING_FOR_FIRST_TIME`. 

In [8]:
# Delete corrupt images in both datasets.
if EXECUTING_FOR_FIRST_TIME:
  check_and_delete_corrupt_images(DATASET_20_PATH, DATASET_20)
  check_and_delete_corrupt_images(DATASET_200_PATH, DATASET_200)

A continuación, visualicemos algunas imágenes de las aves que se se encuentran en el conjunto de datos. Por ejemplo, 5 imágenes aleatorias de gorriones.

![Gorriones muestra](./img_for_docs/show_random_birds.png)

Como podemos ver, las imágenes pueden tener diferentes dimensiones y orientación, lo cual deberemos tener en cuenta a la hora de procesar el conjunto de datos. Aunque lo trataremos más adelante, parace que los pájaros se suelen encontrar en la zona central de la imagen, y bien pueden estar mirando hacia la izquierda o derecha indistintamente. También cabe destacar que, para un ojo no experto, algunas aves de diferentes subespecies presentan grandes similitudes en su apariencia.

In [9]:
if EXECUTING_FOR_FIRST_TIME:
    # Show some images of sparrows, using plt.show() as Visual Studio Code permits you to save images shown, easier file management.
    random.seed(RANDOM_SEED)
    img_sparrows = dict()
    cls_sparrows_total = [k for k in os.listdir(DATASET_200_PATH) if 'sparrow' in k.lower()]
    cls_sparrows = cls_sparrows_total[1::2][:5]
    for dirname in cls_sparrows:
        imgs = list()
        for dp, _, fn in os.walk(os.path.join(DATASET_200_PATH, dirname)):
            imgs.extend(fn)
        img_sparrows[dirname] = imgs
    n_cls = len(cls_sparrows)
    f, ax = plt.subplots(1, n_cls, figsize=(14, 8))

    for i in range(n_cls):
        cls_name = cls_sparrows[random.randint(0, n_cls - 1)]
        n_img = len(img_sparrows[cls_name])
        img_name = img_sparrows[cls_name][random.randint(0, n_img - 1)]
        path_img = os.path.join(os.path.join(DATASET_200_PATH, cls_name), img_name)
        ax[i].imshow(mpimg.imread(path_img))
        ax[i].set_title(cls_name.split('.')[-1].replace('_', ' '),  fontsize=12)

        plt.tight_layout()
    plt.show()

Como mencionábamos anteriormente, las imágenes parecen ser de diferentes dimensiones. 

A continuación definimos una función para comprobar el tamaño de cada imagen en el conjunto de datos y calcular la media, de manera que tengamos información precisa.

In [10]:
def check_images_size_variation(path_to_dataset: str, dataset: str) -> None:
    """
    Computes width and height for each image, results are represented via boxplot. 
    This may take some time
    """
    ds = tv.datasets.ImageFolder(path_to_dataset)
    shapes = [(img.height, img.width) for img, _ in ds]
    heights, widths = [[h for h,_ in shapes], [w for _,w in shapes]]
    print('Average sizes:', *map(np.median, zip(*shapes)))

    fig = plt.figure()
    ax = fig.add_subplot(111)
    bp = ax.boxplot([heights, widths], patch_artist=True)
    ax.set_xticklabels(['height', 'width'])
    ax.set_xlabel('Image sizes for ' + dataset)
    ax.set_ylabel('Pixels')
    plt.show()

In [11]:
# Images are variable in size, let's see how much.
if EXECUTING_FOR_FIRST_TIME:
  check_images_size_variation(DATASET_20_PATH, DATASET_20)
  check_images_size_variation(DATASET_200_PATH, DATASET_200)

La ejecución de la función anterior para ambos conjuntos de datos genera los diagramas de cajas y bigotes que se muestran a continuación.

![Variabilidad tamaños dataset 20](./img_for_docs/image_variability_x20_dataset.png)
![Variabilidad tamaños dataset 200](./img_for_docs/image_variability_x200_dataset.png)

Como podemos comprobar, la mayoría de las imágenes del conjunto de datos reducidos tienen un alto de entre 330 y 430 píxeles, siendo la media 375 píxeles. El ancho por su parte suele ser superior a los 420 píxeles, y la media es cercana a los 500; aunque hay algunos outliers con un ancho menor a los 300 píxeles. Para el conjunto de datos completo la variabilidad en la altura de las imágenes es algo menor, aunque la media se sigue situando en 375 píxeles; el ancho por su parte suele ser de 500 píxeles, aunque se muestran bastantes outliers (círculos por debajo del primer cuartil).

Dado que necesitamos que todas las imágenes tengan las mismas dimensiones, definimos una función que las redimensione (pad) a un tamaño dado, rellenando los píxeles adicionales con un determinado color (sólido negro por defecto). Esta función tiene en cuenta si se va a utilizar dentro de PyTorch o no, dado que la altura y anchura de la imagen se obtienen de forma diferente en función de si es un array de Numpy (para generar las imágenes que mencionaremos a continuación) o si es un tensor de PyTorch.

In [12]:
def pad(img, fill=0, size_max=500, used_for_pytorch=True):
    """
    Pads images to the specified size (height x width). 
    Fills up the padded area with value(s) passed to the `fill` parameter. 
    """
    if used_for_pytorch:
      pad_height = max(0, size_max - img.height)
      pad_width = max(0, size_max - img.width)
    else:
      pad_height = max(0, size_max - img.shape[-2])
      pad_width = max(0, size_max - img.shape[-1])

    pad_top = pad_height // 2
    pad_bottom = pad_height - pad_top
    pad_left = pad_width // 2
    pad_right = pad_width - pad_left
    
    return tv.transforms.functional.pad(img, (pad_left, pad_top, pad_right, pad_bottom), fill=fill)

A continuación, definimos una función para calcular el valor medio de cada píxel dadas las imágenes de un conjunto de datos. Con esta función pretendemos localizar donde se encuentran principalmente los pájaros (con colores significativamente distintos del fondo y entre sí), lo cual podría darnos alguna información de técnicas de procesamiento a utilizar, tipos concretos de redes neuronales que podamos usar o ajustes a sus parámetros.

In [13]:
def average_image_with_padding(path_to_dataset: str) -> None:
    """
    Pad each image so it matches the maximum size of the dataset.
    Calculates the mean of the image, so we can see where data is concentrated
    """
    ds = tv.datasets.ImageFolder(path_to_dataset, transform=tv.transforms.ToTensor())
    img_mean = np.zeros((3, 500, 500))
    for img, _ in ds:
        img = pad(img, used_for_pytorch=False)
        img_mean += img.numpy()

    img_mean = img_mean / len(ds)

    # visualize the average image  
    plt.imshow(np.moveaxis(img_mean, 0, 2))
    plt.show()

In [14]:
# Gonna check where the birds are usually found in the images, as this may give us a clue for later processing.
if EXECUTING_FOR_FIRST_TIME:
    average_image_with_padding(DATASET_20_PATH)
    average_image_with_padding(DATASET_200_PATH)

Las imágenes resultado de la ejecución del código anterior son las siguientes:

![Main Info dataset x20](./img_for_docs/image_main_info_x20_dataset.png)
![Main Info dataset x200](./img_for_docs/image_main_info_x200_dataset.png)

Tanto para el conjunto pequeño (izquierda), como para el conjunto completo (derecha) la media presenta valores/colores distintos en la parte central de la imagen, por lo que podemos afirmar que los pájaros se suelen encontrar en la parte central de las imágenes. Esto parece concordar con lo que podíamos observar en un primer momento al ver algunas imágenes del conjunto de datos.

El hecho de que los pájaros suelan encontrarse en el centro de las imágenes puede llevar al modelo a enfocarse principalmente en esa área, mientras ignora objetos relevantes ubicados en otras partes de la imagen. Intentaremos darle solución a este hecho algo más adelante con el aumento de datos.

## Preparación de los datos

En esta sección prepararemos los datos y definiremos las estructuras necesarias para llevar a cabo los entrenamientos de los diferentes modelos de clasificación.

Llegados a este momento cabe destacar que, aunque se han descargado las imágenes del fichero proporcionado por el profesorado de la asignatura y se sigue su estructura de directorios, se han descargado algunos ficheros adicionales de la web original de [Caltech Library](https://data.caltech.edu/records/65de6-vp158) donde se publicó el conjunto de datos. Concretamente, estos ficheros son `train_test_split` y `bounding_boxes`. 

El primero de ellos define si una imagen se emplea para el conjunto de entrenamiento (1) o para el conjunto de test (0), mientras que el segundo define las coordenadas de las bounding boxes que contienen al ave en cada imagen. Finalmente no hemos empleado las bounding boxes dado que no son estrictamente necesarias para un problema de clasificación, pero se mantiene el código asociado a su procesamiento dado que podrían ser útiles para modelos como YOLO, capaces de realizar tanto clasificación como segmentación, y que necesitan de esta información.

Tanto estos ficheros, como el fichero `images`, han sido renombrados para mostrar al final el número de clases (y por tanto conjunto de datos al que hacen referencia, _x20 o _x200). Si se hace uso de los ficheros adjuntos en esta entrega, no necesita realizar ninguna gestión adicional; si por el contrario ha usado los descargados, por favor renómbrelos a `images_x200.txt`, `train_test_split_x200.txt` y `bounding_boxes_200.txt` (o modifique el código mostrado en las siguientes celdas).

Dado que en principio sólo se nos proporcionan los ficheros para el conjunto de datos completo, necesitamos generar los equivalentes para el conjunto de datos pequeño. Esto no es estrictamente necesario, pues bastaría con coger las 1.155 (número de imágenes del conjunto pequeño) primeras líneas de los ficheros disponibles. Para facilitar el procesamiento y favorecer el "separation of concerns", buena práctica en informática, se generan unos nuevos ficheros con la función y código de las dos siguientes celdas.

In [15]:
def save_first_lines(input_file, output_file, num_lines):
    """
    Given an input file, it stores the first num_lines in the given output_file
    """
    with open(input_file, 'r') as input_f:
        lines = input_f.readlines()[:num_lines]

    with open(output_file, 'w') as output_f:
        output_f.writelines(lines)

In [16]:
# Original dataset provided us with data for 200 classes dataset, we are gonna generated the equivalent files for the data x20.
# We could just use the first 1155 lines of the other files, but, for separation of concerns and easier processing, we are generating the new files
if EXECUTING_FOR_FIRST_TIME:
    data_file_paths =  get_filepaths(DATASET_20_PATH)
    save_first_lines(
        os.path.join(os.path.join(os.getcwd(), 'data additional'), 'train_test_split_x200.txt'),
        os.path.join(os.path.join(os.getcwd(), 'data additional'), 'train_test_split_x20.txt'),
        len(data_file_paths))
    
    save_first_lines(
        os.path.join(os.path.join(os.getcwd(), 'data additional'), 'images_x200.txt'),
        os.path.join(os.path.join(os.getcwd(), 'data additional'), 'images_x20.txt'),
        len(data_file_paths))

    save_first_lines(
        os.path.join(os.path.join(os.getcwd(), 'data additional'), 'bounding_boxes_x200.txt'),
        os.path.join(os.path.join(os.getcwd(), 'data additional'), 'bounding_boxes_x20.txt'),
        len(data_file_paths))

Llegados a este punto, disponemos de todos los elementos necesarios para crear el cargador de datos que utilizaremos posteriormente para entrenar los modelos de clasificación.

Utilizaremos como cargador de datos la clase `DatasetBirds` que se define a continuación, y que hereda de la clase `ImageFolder` y consecuentemente de `DatasetFolder` de PyTorch.

Necesitaremos obligatoriamente de la ruta a los datos, las transformaciones que se aplicarán a los datos (las definimos posteriormente), que cargador usar (por defecto), si estamos en entrenamiento o test y cómo chequear si un fichero es corrupto. Nótese que esta última función la marcamos como `None` dado que ya hemos elimiando los ficheros corruptos de antemano.

Adicionalmente, se añaden dos parámetros opcionales y extra, que indican si usar bounding boxes (marcado a `False` dado que al final no las hemos usado) y el nombre del dataset a utilizar.

In [17]:
class DatasetBirds(tv.datasets.ImageFolder):
    """
    Wrapper for the CUB-200-2011 datasets. 
    Method DatasetBirds.__getitem__() returns tuple of image and its corresponding label.    
    """
    def __init__(self, 
                 root, 
                 transform=None, 
                 target_transform=None, 
                 loader=tv.datasets.folder.default_loader,
                 is_valid_file=None, 
                 train=True,
                 bboxes=False,
                 dataset=None):
      
      if os.path.exists(root): 
        img_root = root
      else:
        raise FileNotFoundError(f"The path '{root}' does not exist.")

      if not img_root.endswith(dataset):
        raise Exception(f"Mismatch between the path to the dataset and the type of dataset specified.")

      super(DatasetBirds, self).__init__(
          root=img_root, 
          transform=None, 
          target_transform=None,
          loader=loader, 
          is_valid_file=is_valid_file
      )

      self.transform_ = transform
      self.target_transform_ = target_transform
      self.train = train

      # Getting the number of classes from the dataset name
      match = re.search(r'\d+$', dataset)
      if match:
        num_classes = match.group(0)
      else:
        raise Exception(f"Dataset type specified doesn't contain the number of classes at the end of the string. It should end in _x<number_classes>")

      train_test_split_filename = 'train_test_split_x' + num_classes + '.txt'
      images_filename = 'images_x' + num_classes + '.txt'
      bounding_boxes_filename = 'bounding_boxes_x' + num_classes + '.txt'

      path_to_splits = os.path.join(os.path.join(os.getcwd(), 'data additional'), train_test_split_filename)
      indices_to_use = list()
      with open(path_to_splits, 'r') as in_file:
          for line in in_file:
              idx, use_train = line.strip('\n').split(' ', 2)
              if bool(int(use_train)) == self.train:
                  indices_to_use.append(int(idx))
      
      path_to_index = os.path.join(os.path.join(os.getcwd(), 'data additional'), images_filename)
      filenames_to_use = set()
      with open(path_to_index, 'r') as in_file:
          for line in in_file:
              idx, fn = line.strip('\n').split(' ', 2)
              if int(idx) in indices_to_use:
                  filenames_to_use.add(fn)

      # Finally not using the bounding boxes, left in case we want to use them later for some model like YOLO.
      if bboxes:
          path_to_bboxes = os.path.join(os.path.join(os.getcwd(), 'data additional'), bounding_boxes_filename)
          bounding_boxes = list()
          with open(path_to_bboxes, 'r') as in_file:
              for line in in_file:
                  idx, x, y, w, h = map(lambda x: float(x), line.strip('\n').split(' '))
                  if int(idx) in indices_to_use:
                      bounding_boxes.append((x, y, w, h))

          self.bboxes = bounding_boxes
      else:
          self.bboxes = None

      # / may be changed to \ depending on the OS, we are asuming Linux.
      img_paths_cut = {'/'.join(img_path.rsplit('/', 2)[-2:]): idx for idx, (img_path, lb) in enumerate(self.imgs)}
      imgs_to_use = [self.imgs[img_paths_cut[fn]] for fn in filenames_to_use]

      _, targets_to_use = list(zip(*imgs_to_use))

      self.imgs = self.samples = imgs_to_use
      self.targets = targets_to_use

    def __getitem__(self, index):
        sample, target = super(DatasetBirds, self).__getitem__(index)

        if self.bboxes is not None:
            width, _ = sample.width, sample.height
            x, y, w, h = self.bboxes[index]
            
            # Some models,as YOLO, use different bbox formats. 
            scale_resize = 500 / width
            scale_resize_crop = scale_resize * (375 / 500)

            x_rel = scale_resize_crop * x / 375
            y_rel = scale_resize_crop * y / 375
            w_rel = scale_resize_crop * w / 375
            h_rel = scale_resize_crop * h / 375

            target = torch.tensor([target, x_rel, y_rel, w_rel, h_rel])

        if self.transform_ is not None:
            sample = self.transform_(sample)
        if self.target_transform_ is not None:
            target = self.target_transform_(target)

        return sample, target


Esencialmente, el constructor:
- Comprueba que el directorio donde están los datos concuerda con el nombre del conjunto de datos. No tendría sentido decir que vamos a usar el conjunto pequeño con los ficheros del grande, o viceversa.
- Asigna las transformaciones que van a aplicarse sobre los datos para realizar un aumento de datos.
- Marca si estamos en entrenamiento o test.
- Lee de los correspondientes ficheros las rutas a las imágenes y cuáles se van a usar en entrenamiento y cuáles en test.
- Genera para cada imagen una tupla con la ruta a dicha imagen y cual es la clase (derivada del nombre de la carpeta donde se ubica).
- Asigna las imágenes a usar y los `targets`.

También tenemos que sobreescribir el método indexador, u operador []. Dado un índice, debemos poder obtener la imagen (y clase asociada) en dicha posición, tras haber aplicado las transformaciones correspondientes.

Nótese que aunque finalmente no se utilizan bounding boxes se mantiene la funcionalidad asociada implementada. Téngase también en cuenta que existen varios formatos para representar bounding boxes. En los ficheros de este conjunto de datos viene representadas por coordenadas y distancias absolutas, pero podrían seguir otros formatos.

## Aumentando los datos

En este momento, definimos las transformaciones de las que haremos uso para aumentar los datos del conjunto de imágenes del que disponemos. 

Como se mencionaba anteriormente, necesitamos que todas las imágenes tengan las mismas dimensiones. Para conseguirlo, definimos una transformación de PyTorch (`max_padding`) haciendo uso de la función de padding que ya se comentó durante el análisis exploratorio. En esta ocasión, en lugar de rellenar con negro sólido se rellena con el color medio para el conjunto de datos de ImageNet (llevado al rango \[0, 255\]). Esta decisión se fundamenta en que en apartados posteriores partiremos del entrenamiento realizado haciendo uso de ciertos modelos sobre el conjunto de datos de ImageNet (compuesto por 1000 clases, entre las que hay algunas de animales).

In [18]:
# Images have different sizes, we are gonna use padding to the max dimensions in the dataset.
fill = tuple(map(lambda x: int(round(x * 256)), (0.485, 0.456, 0.406))) # Mean pixels value of each pixel for ImageNet.
max_padding = tv.transforms.Lambda(lambda x: pad(x, fill=fill))  

Otras transformaciones que vamos a aplicar durante el entrenamiento de los modelos de clasificación es son el flip horizontal y vertical, el recorte aleatorio y los cambios de brillo, contraste y saturación. 
- El flip horizontal nos permitirá distinguir pájaros independienemente de la orientación a la que mire su pico (consultar imágenes del análisis exploratorio). 
- En este conjunto de datos la mayoría de los pájaros se encuentran posados en alguna superficie, por lo que su orientación es la habitual. Por tanto, podría pensarse que aplicar flip vertical es poco menos que inútil, sin embargo, de encontrarse en vuelo (o de tratar con pájaros como murciélagos) podríamos encontrarlo en otras orientaciones, por lo que nos anticipamos a estas situaciones.
- La mayoría de los pájaros se encuentran en el centro de la imagen, pero podría no ser siempre así. Para conseguir que el modelo realice una mejor generalización realizamos un recorte aleatorio de la imagen.
- Las fotografías podrían no encontrarse siempre perfectamente expuestas (contraste, brillo y saturación diferente a lo esperado, simplificando mucho), por lo que realizamos pequeñas variaciones de los mismos. No se aplican grandes variaciones bajo la suposición de que el color o tonalidad del pelaje del ave podría ser característico de una determinada especie. Recordamos una vez más que no somos expertos en la materia.

Finalmente, convertimos a tensor y normalizamos con estadísticas propias de ImageNet.

In [19]:
# Mayority of samples show the bird in the middle of image. 
# Test images will be center-cropped by 375x375 pixeles, and normalized by ImageNet's statistics 
transforms_train = tv.transforms.Compose([
   max_padding,
   tv.transforms.RandomOrder([
       tv.transforms.RandomCrop((375, 375)),
       tv.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
       tv.transforms.RandomHorizontalFlip(),
       tv.transforms.RandomVerticalFlip()
   ]),
   tv.transforms.ToTensor(),
   tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
transforms_eval = tv.transforms.Compose([
   max_padding,
   tv.transforms.CenterCrop((375, 375)),
   tv.transforms.ToTensor(),
   tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]) 

Para la evaluación nos limitaremos a realizar un recorte en el centro de la imagen (pues la mayoría de los pájaros se encuentran en esta posición) y a normalizar de acuerdo a valores propios de ImageNet.

## Creando los conjuntos de entrenamiento, validación y test

Lo siguiente que deberemos hacer es gener los conjuntos de entrenamiento, validación y test a partir de la clase de carga de datos definida anteriormente, a la que se le pasa como parámetros la ruta a los datos, el dataset a emplear y las tranformaciones que deseamos aplicar. Los splits los creamos haciendo uso de funciones de la librería Scikit-Learn. A partir de estos splits generamos los índices para entrenamiento y validación. Este proceso se replica para el conjunto de datos pequeño y para el conjunto completo.

In [20]:
# Create training, validation and test according to loaded splits.
ds_x20_train = DatasetBirds(DATASET_20_PATH, transform=transforms_train, train=True, dataset=DATASET_20)     
ds_x20_val = DatasetBirds(DATASET_20_PATH, transform=transforms_eval, train=True, dataset=DATASET_20)
ds_x20_test = DatasetBirds(DATASET_20_PATH, transform=transforms_eval, train=False, dataset=DATASET_20)

splits_x20 = skms.StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=RANDOM_SEED)
idx_train_x20, idx_val_x20 = next(splits_x20.split(np.zeros(len(ds_x20_train)), ds_x20_train.targets))

In [21]:
# Create training, validation and test according to loaded splits 
ds_x200_train = DatasetBirds(DATASET_200_PATH, transform=transforms_train, train=True, dataset=DATASET_200)
ds_x200_val = DatasetBirds(DATASET_200_PATH, transform=transforms_eval, train=True, dataset=DATASET_200)
ds_x200_test = DatasetBirds(DATASET_200_PATH, transform=transforms_eval, train=False, dataset=DATASET_200)

splits_x200 = skms.StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=RANDOM_SEED)
idx_train_x200, idx_val_x200 = next(splits_x200.split(np.zeros(len(ds_x200_train)), ds_x200_train.targets))

También deberemos definir los hiperparámetros con los que vamos a realizar el entrenamiento de la red. Estos deben definirse en función de si usamos Google Colab o no, y en función del hardware de nuestro equipo.

Si usamos el plan gratuito de Google Colab disponemos de 2 hilos de CPU y una tarjeta gráfica NVIDIA TESLA T4, con 16 GB de memoria VRAM. Esto nos permitiría especificar un tamaño de lote de 32 imágenes; necesitando 11 GB de la memoria de la GPU para realizar el entrenamiento. No se ha probado demasiado dado que finalmente las ejecuciones se han realizado en la máquina local, pero esta GPU podría llegar a albergar lotes de tamaño 40 o incluso 48.

De usar nuestra máquina local, debemos definir los parámetros en función a las características concretas de nuestro hardware. En lo relativo al número de hilos de CPU se ha definido que se usen como mínimo 2, y como máximo tantos como disponga el procesador-2 (para no saturarlo por si tuvieramos que hacer otra cosa mientras). 

Dada nuestra tarjeta gráfica, NVIDIA GTX 1080, con 8 GB de VRAM, podemos llegar a emplear lotes de tamaño 24 o inferior. De usar tamaño de lote 24 y ResNet-50 se necesitan unas 7.4 GB de VRAM, las cuales normalmente están disponibles, pero podría no ser el caso de usar varias pantallas o emplear otros programas durante el entrenamiento (lo cual no sería recomendable). Con tamaño de lote 16 se usan unas 5.5 GB de RAM, y no se tiene problema alguno durante la ejecución del entrenamiento del modelo ResNet-50. De tener tarjetas gráficas de menor capacidad deberá rebajarse el tamaño de lote, o usar Google Colab de ser posible. El modelo MobileNetV3 gasta menos memoria y no se tiene problema para ejecutarlo.

In [22]:
# Set hyper-parameters.
# Not recommended to use more workers than CPU threads
if USING_COLAB: 
  params = {'batch_size': 32, 'num_workers': 2} # Colab has CPU with 2 threads and TESLA T4.
else:
  params = {'batch_size': 16, 'num_workers': max(mp.cpu_count()-2, 2)}  
  # Local machine has 12 CPU Cores and 1 GPU (GTX 1080) with 8 GB of VRAM. 

Finalmente, definimos las estructuras encargadas de cargar los datos a partir de los propios conjuntos de datos que acabamos de definir, el sampler y los hiperparámetros que acabamos de comentar.  Como sampler se usa el `SubsetRandomSampler` para elegir un subconjunto de los datos de forma aleatoria, el cual podría ser ciertamente numeroso dado el tamaño del conjunto de datos tras haber aplicado las transformaciones para el aumento de datos.

Este proceso se realiza tanto para el conjunto pequeño como para el conjunto completo.

In [23]:
# Instantiate data loaders
train_loader_x20 = torch.utils.data.DataLoader(
   dataset=ds_x20_train,
   sampler=torch.utils.data.SubsetRandomSampler(idx_train_x20),
   **params
)
val_loader_x20 = torch.utils.data.DataLoader(
   dataset=ds_x20_val,
   sampler=torch.utils.data.SubsetRandomSampler(idx_val_x20),
   **params
)
test_loader_x20 = torch.utils.data.DataLoader(dataset=ds_x20_test, **params)

train_loader_x200 = torch.utils.data.DataLoader(
   dataset=ds_x200_train,
   sampler=torch.utils.data.SubsetRandomSampler(idx_train_x200),
   **params
)
val_loader_x200 = torch.utils.data.DataLoader(
   dataset=ds_x200_val,
   sampler=torch.utils.data.SubsetRandomSampler(idx_val_x200),
   **params
)
test_loader_x200 = torch.utils.data.DataLoader(dataset=ds_x200_test, **params)

## Clasificador ResNet-50 "desde cero"

Tras haber definido las particiones para entrenamiento, validación y test, debemos definir un modelo de aprendizaje profundo que las use.

Podríamos desarrollar una arquitectura propia desde cero, pero en su lugar, vamos a utilizar alguna arquitectura ya implementada en PyTorch. Este paquete software proporciona una larga lista de arquitecturas, como AlexNet, EfficientNet, MobileNet, VGG, entre otras. 

Tras haber consultado las especificaciones y algunas estadísticas de cada una de ellas en la documentación de PyTorch, elegimos utilizar la familia de arquitecturas ResNet, en tanto que proporcionan una buena precisión sobre el conjunto de datos ImageNet a cambio de una potencia computacional menor que otras alternativas consultadas, y en principio asumible por el hardware del que disponemos.

En PyTorch disponemos de 5 implementaciones de ResNet, con 18, 34, 50, 101 y 152 capas respectivamente. Elegimos hacer uso de la versión con 50 capas en tanto que parece ofrecer la mejor relación entre precisión y coste computacional. La versión con 34 capas ofrece cerca de un 7% menos de precisión con un coste computacional sólo 11% menor, mientras que las versiones más complejas ofrecen 1.02% y 1.4% más de precisión con un coste computacional casi del doble y triple, respectivamente.

La arquitectura ResNet fue propuesta por Kaiming He, Xiangyu Zhang, Shaoqing Ren y Jian Sun (Microsoft Research) en 2015 en un intento de solucionar el problema del "desvanecimiento del gradiente", y aunque el objetivo de esta práctica no es el análisis teórico de la arquitectura sí que destacaremos su principal innovación. 

Esta arquitectura incorpora las llamadas "capas residuales", de ahí el nombre. En lugar de intentar aprender directamente la representación deseada en cada capa, las conexiones residuales permiten a la red aprender las diferencias entre la salida deseada y la salida actual de la capa. Esto se logra añadiendo conexiones directas, llamadas "saltos", que saltan una o más capas y habilitan que el gradiente fluya por la red evitando su desvanecimiento. 

In [24]:
model_x20 = tv.models.resnet50(num_classes=20).to(DEVICE)
model_x200 = tv.models.resnet50(num_classes=200).to(DEVICE)
if EXECUTING_FOR_FIRST_TIME:
    print(model_x20) # Print default version (groupped with all info)
    torchsummary.summary(model_x20, input_size=(3,375,375)) # Print the summary formatted, it is the same for both models.

Tras haber instanciado el modelo, podríamos consultar las capas concretas por las que está formado. Dada la gran extensión de la salida no se mostrará en este cuaderno, pero sí que destacaremos que tiene 23.549.012 parámetros entreanables y que ocupa 907.08 MB en memoria. 

También cabe mencionar que la arquitectura ResNet implementada en PyTorch es distinta de la propuesta originalmente por los autores en 2015. Si acudimos a la documentación de PyTorch, esta nos indica que "the bottleneck of TorchVision places the stride for downsampling to the second 3x3 convolution while the original paper places it to the first 1x1 convolution. This variant improves the accuracy and is known as ResNet V1.5".



### Entrenamiento y evaluación

Antes de comenzar con el entrenamiento de nuestro modelo, definiremos tres funciones que necesitaremos más tarde.

La primera de ellas simplemente genera una cadena de texto en función del tipo de modelo que estemos entrenando (pues tendremos más en esta práctica a parte del descrito en esta sección), de cara al nombre con el que guardaremos los ficheros con los pesos resultantes del entrenamiento. Las otras dos funciones las emplearemos para generar gráficos a partir de los valores de pérdida y precisión que obtengamos tras entrenar el modelo.

In [25]:
def get_model_desc(num_classes=200, pretrained=False, model=None):
    """
    Generates description string. Used for generated the name of the model when saving it to disk. 
    """
    desc = list()
    if model == 'resnet50':
        desc.append('ResNet50')
    elif model == 'mobilenet':
        desc.append('MobileNet')
    elif model == 'efficientnet':
        desc.append('EfficientNet')

    if pretrained:
        desc.append('Transfer')
    else:
        desc.append('Baseline')

    return '_'.join(desc)

In [26]:
def graph_loss_function(train_loss, val_loss):
    """
    Graphs the evolution of the loss function. 
    Requires both the loss of the training and validation set.
    """
    x = range(1,len(train_loss)+1)

    fig, ax = plt.subplots()
    plt.plot(x, train_loss, label='Entrenamiento')
    plt.plot(x, val_loss, label='Validación')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.title('Evolución de la función de pérdida')
    plt.legend()
    plt.show()

def graph_accuracy_function(train_acc, val_acc, max_value = 1.01):
    """
    Graphs the evolution of the accuracy function.
    Requires both the accuracy of the training and validation set.
    As optional parameter, the maximum value of the y axis can be specified. It has to be a float a bit higher than the strict limit.
    """
    x = range(1,len(train_acc)+1)

    fig, ax = plt.subplots()
    plt.plot(x, train_acc, label='Entrenamiento')
    plt.plot(x, val_acc, label='Validación')
    plt.xlabel('Épocas')
    plt.ylabel('Precisión')
    plt.yticks(np.arange(0, max_value, 0.05))
    plt.title('Evolución de la precisión')
    plt.legend()
    plt.show()

Para poder entrenar el modelo necesitaremos definir un optimizador y un "scheduler" (actualizador de la tasa de aprendizaje) .

El optimizador se encargará de actualizar los parámetros del modelo en función del gradiente calculado durante la retropropagación; mientras que el actualizador de tasa de aprendizaje controlará cómo se ajusta la tasa de aprendizaje a medida que avanza el entrenamiento (reduciendolo exponencialmente por un factor `gamma` tras cada época). 

Siguiendo las recomendaciones y experimentos llevados a cabo en la asignatura "Inteligencia Computacional" de estos mismos estudios de máster, decidimos hacer uso del optimizador ADAM al proporcionar este un mejor desempeño que alternativas como SGD o RMSprop. Los valores para las tasas del `learning rate` y `gamma` se toman de la bibliografía consultada, pero concuerdan con lo que vimos en la asignatura anteriormente mencionada durante el pasado cuatrimestre.

In [27]:
optimizer_x20 = torch.optim.Adam(model_x20.parameters(), lr=1e-3)
scheduler_x20 = torch.optim.lr_scheduler.ExponentialLR(optimizer_x20, gamma=0.95)

optimizer_x200 = torch.optim.Adam(model_x200.parameters(), lr=1e-3)
scheduler_x200 = torch.optim.lr_scheduler.ExponentialLR(optimizer_x200, gamma=0.95)

Para evitar la repetición de código, ya que entrenaremos varios modelos durante esta práctica, definimos dos funciones encargadas de realizar el entrenamiento en sí mismo y posteriormente el test del modelo generado.

La función de entrenamiento requerirá del modelo, optimizador y "scheduler" a utilizar, así como los objetos encargados de cargar los datos de los conjuntos de entrenamiento y validación. También se necesita del tipo de conjunto de datos a utilizar y el flag `pretrained` para generar el nombre del fichero en el que se guardarán los pesos. Como no podía ser de otra manera, también necesitaremos especificar el número de épocas durante las cuales deseamos entrenar. 

Esta función nos devolverá la ruta al fichero en el que se han guardado los pesos, los cuales serán los mejores obtenidos durante el entrenamiento, y cuatro listas con los valores de la función de pérdida y precisión para el conjunto de entrenamiento y validación en cada época. Los valores de estas cuatro listas las utilizaremos para generar algunas gráficas.

Por su parte, la función de test solamente requiere del modelo a testear, la ruta desde la cual cargar los pesos que se deseen usar y el objeto encargado de cargar las imágenes que van a ser usadas para el test. Esta función simplemente devolverá la precisión obtenida por el modelo para el conjunto de test especificado.

In [28]:
def train_validate_model(model, scheduler, optimizer,
                                train_loader, val_loader, dataset: str, 
                                num_epochs: int,  modelname: str, pretrained: bool = False) -> Tuple[str, list, list, list, list]:
  """
  Function to train and validate a model given a type of dataset
  It save the weights in a .pt file ans returns the path to it
  Returns:
    - best_snapshot_path: path to the best model snapshot
    - all_train_loss: list of all training losses mean values
    - all_train_acc: list of all training accuracies mean values
    - all_val_loss: list of all validation losses mean values
    - all_val_acc: list of all validation accuracies mean values
  """

  # Check consistency of dataset type
  match = re.search(r'\d+$', dataset)
  if match:
    num_classes = match.group(0)
  else:
    raise Exception(f"Dataset type specified doesn't contain the number of classes at the end of the string. It should end in _x<number_classes>")

  model_desc = get_model_desc(int(num_classes), pretrained, modelname)
  best_snapshot_path: str = ""
  best_val_acc = 0.0
  all_train_loss = list() # Mean values
  all_train_acc = list()
  all_val_loss = list()   # Mean values
  all_val_acc = list()  


  for epoch in range(num_epochs):
    model.train()
    train_loss = list()
    train_acc = list()
    for batch in train_loader:
        x, y = batch
        
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        
        optimizer.zero_grad()
        
        # predict bird species
        y_pred = model(x)

        # calculate the loss
        loss = F.cross_entropy(y_pred, y)
        # calculate the accuracy
        acc = skmetrics.accuracy_score([val.item() for val in y], [val.item() for val in y_pred.argmax(dim=-1)])
        
        # backprop & update weights 
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())
        train_acc.append(acc)

    all_train_loss.append(np.mean(train_loss))
    all_train_acc.append(np.mean(train_acc))

    # validate the model
    model.eval()
    val_loss = list()
    val_acc = list()
    with torch.no_grad():
        for batch in val_loader:
            x, y = batch
            x = x.to(DEVICE)
            y = y.to(DEVICE)
            y_pred = model(x)

            # calculate the loss
            loss = F.cross_entropy(y_pred, y)
            # calculate the accuracy
            acc = skmetrics.accuracy_score([val.item() for val in y], [val.item() for val in y_pred.argmax(dim=-1)])

            val_loss.append(loss.item())
            val_acc.append(acc)

        all_val_acc.append(np.mean(val_acc))
        all_val_loss.append(np.mean(val_loss))
            
        # save the best model snapshot
        current_val_acc = all_val_acc[-1]
        if current_val_acc > best_val_acc:
            print("New best accuracy {:.5f} > {:.5f} at epoch {}".format(current_val_acc, best_val_acc, epoch+1))
            if best_snapshot_path != "":
                os.remove(best_snapshot_path)

            best_val_acc = current_val_acc
            best_snapshot_path = os.path.join(OUT_DIR, f'model_{model_desc}_x{num_classes}_ep={epoch+1}_acc={best_val_acc}.pt')

            torch.save(model.state_dict(), best_snapshot_path)

    # Adjust the learning rate
    scheduler.step()


    if (epoch == 0) or ((epoch + 1) % 3 == 0 or (epoch + 1) == num_epochs):
        print('Epoch {} |> Train. loss: {:.4f} | Val. loss: {:.4f}'.format(
            epoch + 1, np.mean(train_loss), np.mean(val_loss))
        )
        print('Epoch {} |> Train. acc.: {:.4f} | Val. acc.: {:.4f}'.format(
            epoch + 1, np.mean(train_acc), np.mean(val_acc))
        )

  return best_snapshot_path, all_train_loss, all_train_acc, all_val_loss, all_val_acc

def test_model(model, test_loader, best_snapshot_path):
  """
  Function to test a model given the test loader and the path to a snapshot of the model (theorically the best one)
  Returns the accuracy
  """

  model.load_state_dict(torch.load(best_snapshot_path, map_location=DEVICE))
        
  true = list()
  pred = list()
  with torch.no_grad():
      for batch in test_loader:
          x, y = batch
          x = x.to(DEVICE)
          y = y.to(DEVICE)
          y_pred = model(x)

          true.extend([val.item() for val in y])
          pred.extend([val.item() for val in y_pred.argmax(dim=-1)])

  
  test_accuracy = skmetrics.accuracy_score(true, pred)
  print('\nTest accuracy: {:.3f}'.format(test_accuracy))
  return test_accuracy

Dado que los entrenamientos pueden tomar un tiempo considerable en función de la arquitectura empleada, conjunto de datos y número de épocas; antes de cada entrenamiento definiremos la ruta a los mejores resultados que hemos obtenido. Téngase la precaución de descargar los ficheros con los pesos (cuyos enlaces se encontrarán al final de este documento) y colocarlos en la carpeta `dataset/results`. No se comprueba si la ruta es válida, por lo que si se ejecutan sólamnete los test sin entrenar y sin disponer de los ficheros se producirá un error.

Téngase en cuenta que de ejecutar el entrenamiento de un modelo se tomará la ruta al fichero con los pesos que mejores resultados han proporcionado durante dicho entrenamiento, ignorandose aquellos ficheros de los que pudieramos disponer en la carpeta y que eventualmente podrían ser mejores.

In [29]:
BEST_BASELINE_RESNET_X20_PATH = os.path.join(os.path.join(os.getcwd(), 'results'), 'model_ResNet50_Baseline_x20_ep=47_acc=0.5625.pt')
BEST_BASELINE_RESNET_X200_PATH = os.path.join(os.path.join(os.getcwd(), 'results'), 'model_ResNet50_Baseline_x200_ep=30_acc=0.23355263157894737.pt')

Finalmente, ejecutamos el entrenamiento del modelo para el conjunto de datos pequeño y graficamos la evolución de la función de pérdida y precisión durante el mismo.

Los resultados que se muestran en este reporte son los mejores obtenidos, aunque se discutirán resultados obtenidos con otras configuraciones distitnas a la mencionada.

In [30]:
train_loss_baseline_resnet_x20, train_acc_baseline_resnet_x20, val_loss_baseline_resnet_x20, val_acc_baseline_resnet_x20 = list(), list(), list(), list()

if TRAIN_BASELINE_MODELS and TRAIN_SMALL_MODELS:
  BASELINE_RESNET_X20_RESULTS = train_validate_model(model_x20, scheduler_x20, optimizer_x20, train_loader_x20, val_loader_x20, DATASET_20, 70, 'resnet50')
  BEST_BASELINE_RESNET_X20_PATH = BASELINE_RESNET_X20_RESULTS[0]
  train_loss_baseline_resnet_x20, train_acc_baseline_resnet_x20, val_loss_baseline_resnet_x20, val_acc_baseline_resnet_x20 = BASELINE_RESNET_X20_RESULTS[1:]

if TEST_SMALL_MODELS:
  baseline_x20_accuracy = test_model(model_x20, test_loader_x20, BEST_BASELINE_RESNET_X20_PATH)

In [31]:
if TRAIN_BASELINE_MODELS and TRAIN_SMALL_MODELS and GENERATE_GRAPHS:
    graph_loss_function(train_loss_baseline_resnet_x20, val_loss_baseline_resnet_x20)
    graph_accuracy_function(train_acc_baseline_resnet_x20, val_acc_baseline_resnet_x20, max_value=0.81)

Los mejores resultados han sido obtenidos empleando un `batch_size=16` y las técnicas de aumento de datos mencionadas anteriormente. En estas condiciones, conseguimos una precisión sobre el conjunto de test de 0.47 tras 70 épocas de entrenamiento y 16 minutos y medio de ejecución. Las gráficas con la evolución de la función de pérdida y precisión para entrenamiento y validación se muestra a continuación.

![Baseline x20 loss function](./img_for_docs/baseline_loss_resnet_x20.png)
![Baseline x20 acc function](./img_for_docs/baseline_acc_resnet_x20.png)

A partir de la gráfica de la función de pérdida podemos intuir que se produce un "overfitting" o sobre-ajuste del modelo al conjunto de entrenamiento, pues el valor de dicha función no para de descender para el conjunto de entrenamiento, pero parece estabilizarse dentro de un rango para el conjunto de validación. Algo similar se puede observar en la gráfica que ilustra la evolución de la precisión. 

Los resultados obtenidos son ciertamente pobres, pues el clasificador falla en más ocasiones de las que acierta. 

Se ha probado también a ejecutar entrenamientos con `batch_size=16` prescindiendo del aumento de datos que conseguimos al variar brillo, contraste y saturación; en cuyo caso se ha obtenido una precisión máxima de 0.45. También se han ejecutado experimentos con `batch_size=24`, con y sin variación de brillo, contraste y saturación. En esta última casuística se han conseguido resultados peores, con precisiones que oscilaban entre 0.32 y 0.38, y no se consiguió una reducción en el tiempo de entrenamiento.

Los resultados obtenidos para el conjunto de datos pequeños no son demasiado esperanzadores, pero aun así decidimos ejecutar el entrenamiento para el conjunto de datos completo.

In [32]:
train_loss_baseline_resnet_x200, train_acc_baseline_resnet_x200, val_loss_baseline_resnet_x200, val_acc_baseline_resnet_x200 = list(), list(), list(), list()

if TRAIN_BASELINE_MODELS and TRAIN_BIG_MODELS:
  BASELINE_RESNET_X200_RESULTS = train_validate_model(model_x200, scheduler_x200, optimizer_x200, train_loader_x200, val_loader_x200, DATASET_200, 30, 'resnet50')
  BEST_BASELINE_RESNET_X200_PATH = BASELINE_RESNET_X200_RESULTS[0]
  train_loss_baseline_resnet_x200, train_acc_baseline_resnet_x200, val_loss_baseline_resnet_x200, val_acc_baseline_resnet_x200 = BASELINE_RESNET_X200_RESULTS[1:]

if TEST_BIG_MODELS: 
  baseline_x200_accuracy = test_model(model_x200, test_loader_x200, BEST_BASELINE_RESNET_X200_PATH)

In [33]:
if TRAIN_BASELINE_MODELS and TRAIN_BIG_MODELS and GENERATE_GRAPHS:
    graph_loss_function(train_loss_baseline_resnet_x200, val_loss_baseline_resnet_x200)
    graph_accuracy_function(train_acc_baseline_resnet_x200, val_acc_baseline_resnet_x200, max_value=0.36)

Los resultados obtenidos son ciertamente desastrosos, pues apenas conseguimos una precisión máxima de 0.2335 para `batch_size=16` y 30 épocas tras un tiempo de ejecución de 69 minutos. Ejecuciones con tamaño de lote de 16, con o sin variación en saturación, brillo e intensidad proporcionan resultados similares (oscilan entre 0.20 y 0.225). Dudamos que con un mayor número de épocas consiguieramos aumentar mucho la precisión, por lo que no se ha probado con más de 30 épocas dado que el tiempo de ejecución comienza a ser "poco asumible".

Las gráficas con la evolución de la función de pérdida y precisión para entrenamiento y validación se muestran a continuación, aunque no realicemos una discusión sobre las mismas.

![Baseline x200 loss function](./img_for_docs/baseline_loss_resnet_x200.png)
![Baseline x200 acc function](./img_for_docs/baseline_acc_resnet_x200.png)

Los resultados obtenidos en este apartado eran esperables, pues apenas disponemos de imágenes de cada clase como para entrenar a un clasificador "desde cero". 

En el siguiente apartado realizaremos un proceso de transferencia de aprendizaje (transfer learning o fine tuning) con el objetivo de conseguir un modelo útil que proporcione mejores decisiones.

## Clasificador ResNet-50 a partir de pesos pre-entrenados

Como discutíamos anteriormente, si entrenamos un modelo desde cero con un número reducido de imágenes (incluso si empleamos aumento de datos) para cada clase estamos avocados a obtener un modelo que va a cometer un gran número de fallos en sus predicciones.

Una posible solución a la problemática anterior es partir de los pesos obtenidos al entrenar un modelo con una gran cantidad de imágenes y realizar un proceso de ajuste para que estos se adecuen a nuestro conjunto de datos. Este enfoque nos permite obtener un buen rendimiento y una buena generalización del modelo utilizando un menor número de datos, especialmente si el conjunto de datos con el que se entrenó el modelo original está relacionado con el conjunto que vamos a emplear para realizar el ajuste. Además, este proceso de transferencia de aprendizaje permite reducir significativamente los tiempos de entrenamiento.

Así pues, vamos a crear una nueva instancia del modelo ResNet-50 indicando que se utilicen pesos pre-entreandos. PyTorch nos proporciona la posibilidad de utilizar los pesos obtenidos al entrenar esta arquitectura de red sobre el conjunto de imágenes ImageNet en dos variantes. Haciendo uso de `ResNet50_Weights.DEFAULT` nos aseguramos utilizar aquellos pesos que proporcionan un mejor desempeño.

ImageNet es un conjunto de datos con imágenes de 1000 clases muy variadas, entre las que se encuentran algunos tipos de animales, y lo que para nosotros es muy interesante y beneficioso, aves. Por tanto, esperamos obtener un rendimiento significativamente mejor que el mostrado en el apartado anterior.

In [34]:
# Using pretrained weights on ImageNet. Default weights may vary on Torch version, currently they are IMAGENET1K_V2
model_pretrained = tv.models.resnet50(weights=tv.models.ResNet50_Weights.DEFAULT).to(DEVICE)
new_optimizer = torch.optim.Adam(model_pretrained.parameters(), lr=1e-4)
new_scheduler = torch.optim.lr_scheduler.ExponentialLR(new_optimizer, gamma=0.95)

In [35]:
BEST_PRETRAINED_RESNET_X20_PATH = os.path.join(os.path.join(os.getcwd(), 'results'), 'model_ResNet50_Transfer_x20_ep=3_acc=0.9375.pt')  
BEST_PRETRAINED_RESNET_X200_PATH = os.path.join(os.path.join(os.getcwd(), 'results'), 'model_ResNet50_Transfer_x200_ep=26_acc=0.8157894736842105.pt') 

Tras haber definido el nuevo modelo, y re-definir el optimizador y "scheduler" en consecuencia, procedemos a realizar el entrenamiento del modelo sobre el conjunto de datos pequeño.

In [36]:
train_loss_pretrained_resnet_x20, train_acc_pretrained_resnet_x20, val_loss_pretrained_resnet_x20, val_acc_pretrained_resnet_x20 = list(), list(), list(), list()

if TRAIN_PRETRAINED_MODELS and TRAIN_SMALL_MODELS:
  PRETRAINED_RESNET_X20_RESULTS = train_validate_model(model_pretrained, new_scheduler, new_optimizer, train_loader_x20, val_loader_x20, DATASET_20, 30, 'resnet50', True)
  BEST_PRETRAINED_RESNET_X20_PATH = PRETRAINED_RESNET_X20_RESULTS[0]
  train_loss_pretrained_resnet_x20, train_acc_pretrained_resnet_x20, val_loss_pretrained_resnet_x20, val_acc_pretrained_resnet_x20 = PRETRAINED_RESNET_X20_RESULTS[1:]
if TEST_SMALL_MODELS:
  pretrained_x20_accuracy = test_model(model_pretrained, test_loader_x20, BEST_PRETRAINED_RESNET_X20_PATH)

In [37]:
if TRAIN_PRETRAINED_MODELS and TRAIN_SMALL_MODELS and GENERATE_GRAPHS:
    graph_loss_function(train_loss_pretrained_resnet_x20, val_loss_pretrained_resnet_x20)
    graph_accuracy_function(train_acc_pretrained_resnet_x20, val_acc_pretrained_resnet_x20)

En esta ocasión, los resultados son muchos mejores que los obtenidos en el apartado anterior, ya que conseguimos una precisión sobre el conjunto de test del 94%. Además, nótese que se ha necesitado un número de épocas bastante menor para conseguir dicho resultado, lo que ha permitido reducir el tiempo de entrenamiento a apenas 7 minutos.

Los mejores resultados se han obtenido haciendo uso de `batch_size=16` y el proceso de aumento de datos completo. Experimentos con este tamaño de lote sin variación en brillo, tono o saturación, y experimentos con `batch_size=24` aportan resultados muy similares; oscilando la precisión sobre el conjunto de test entre 0.91 y 0.932. 

Si observamos las gráficas con la evolución de la función de pérdida y la precisión, podemos ver como el modelo se ajusta perfectamente al conjunto de entrenamiento en apenas dos épocas (train acc sumamente cercana a 1), mientras que la precisión en el conjunto de validación tiende a oscilar entre 0.91 y 0.9375. La precisión sobre el conjunto de validación no ha mejorado desde la tercera época, por lo que podríamos haber detenido el entrenamiento de manera preventiva tras apenas 80 segundos.

![Pretrained x20 loss function](./img_for_docs/pretrained_loss_resnet_x20.png)
![Pretrained x20 acc function](./img_for_docs/pretrained_acc_resnet_x20.png)

Con la gran mejora de rendimiento obtenida para el conjunto de datos pequeño, pasamos a ejecutar el proceso de entrenamiento y test sobre el conjunto de datos completo.

In [38]:
# Reset model, optimizer and scheduler just in case
model_pretrained = tv.models.resnet50(weights=tv.models.ResNet50_Weights.DEFAULT).to(DEVICE)
new_optimizer = torch.optim.Adam(model_pretrained.parameters(), lr=1e-4)
new_scheduler = torch.optim.lr_scheduler.ExponentialLR(new_optimizer, gamma=0.95)

train_loss_pretrained_resnet_x200, train_acc_pretrained_resnet_x200, val_loss_pretrained_resnet_x200, val_acc_pretrained_resnet_x200 = list(), list(), list(), list()

if TRAIN_PRETRAINED_MODELS and TRAIN_BIG_MODELS:
  PRETRAINED_RESNET_X200_RESULTS = train_validate_model(model_pretrained, new_scheduler, new_optimizer, train_loader_x200, val_loader_x200, DATASET_200, 30, 'resnet50', True)
  BEST_PRETRAINED_RESNET_X200_PATH = PRETRAINED_RESNET_X200_RESULTS[0]
  train_loss_pretrained_resnet_x200, train_acc_pretrained_resnet_x200, val_loss_pretrained_resnet_x200, val_acc_pretrained_resnet_x200 = PRETRAINED_RESNET_X200_RESULTS[1:]
if TEST_BIG_MODELS:
  pretrained_x200_accuracy = test_model(model_pretrained, test_loader_x200, BEST_PRETRAINED_RESNET_X200_PATH)

In [39]:
if TRAIN_PRETRAINED_MODELS and TRAIN_BIG_MODELS and GENERATE_GRAPHS:
    graph_loss_function(train_loss_pretrained_resnet_x200, val_loss_pretrained_resnet_x200)
    graph_accuracy_function(train_acc_pretrained_resnet_x200, val_acc_pretrained_resnet_x200)

Como podíamos anticipar, los resultados obtenidos son significativamente mejores a los obtenidos haciendo uso de un modelo entrenado desde cero. 

El mejor resultado obtenido permite al modelo clasificar las aves del conjunto de test con una precisión del 82.1%. El entrenamiento que ha dado lugar a estos pesos ha tenido como tamaño de lote 16 imágenes, y ha consistido en 30 épocas que han tardado en ejecutarse 64 minutos y 38 segundos. 

En este entrenamiento, no ha tenido lugar una mejora significativa de la precisión sobre el conjunto de validación a partir de la época 10, por lo que podríamos haber aplicado mecanismos de detención temprana de entrenamiento y haber concluido este en poco más de 20 minutos. Esto no ha sido así en todas las ejecuciones que hemos realizado, ya que en algunas de ellas se han obtenido ganancias cercanas al 5% más allá de las 20 épocas de entrenamiento.

Para otras ejecuciones con tamaño de lote 24, tanto aplicando aumento de datos derivado de la modificación de brillo, contraste y saturación como sin hacerlo; se han obtenido precisiones sobre el conjunto de test que oscilan entre el 77 y 81%. En este sentido, no se presentan diferencias respecto de usar un tamaño de lote de 16. Los tiempos de ejecución no han sufrido cambios drásticos en función del tamaño de lote, oscilando entre los 62 y 66 minutos aproxima e indistintamente. 


![Pretrained x200 loss function](./img_for_docs/pretrained_loss_resnet_x200.png)
![Pretrained x200 acc function](./img_for_docs/pretrained_acc_resnet_x200.png)

Si observamos las gráficas con la evolución de la función de pérdida y la precisión, podemos ver como el modelo se ajusta perfectamente al conjunto de entrenamiento a partir de la época 9, mientras que la precisión en el conjunto de validación se estabiliza y mejora muy tímidamente a partir de la época 6. Los valores de pérdida siguen unas tendencias similares, no disminuyendo significativamente a partir de la época 10 para entrenamiento y 5 para validación.

Sin embargo, no en todas las ejecuciones se ha seguido el comportamiento descrito en el párrafo anterior. En algunas ocasiones la precisión ya alcanzaba valores cercanos al 75% en la primera o segunda época, de manera que la mejora ha sido aún más rápida y el entrenamiento podría haberse detenido en apenas 6 épocas. En estos casos la precisión final sobre el conjunto de test era cercana al 80%. Análogamente, el descenso del valor de pérdida ha sido aún más pronunciado que en las gráficas mostradas.

En una ocasión concreta, el modelo comenazaba en sus primeras épocas presentando unas precisiones sobre los conjuntos de entrenamiento y validación mucho más reducidas, de entorno al 30-40% durante las 6 primeras épocas. La mejora en la métrica era constante, pero el resultado final fue el peor de los ejecutados, con una precisión del 72%. Esta ejecución empleaba tamaño de lote 16 y todas las transformaciones descritas en el apartado de aumento de datos. No se consiguió replicar esta ejecución, proporcionando las demás valores de precisión en el rango 77-81%, salvo la destacada, que supera el 82%.

## Clasificador EfficientNet

Otra arquitectura, alternativa a ResNet, que parece proporcionar muy buenos resultados según la documentación de PyTorch es EfficientNet. 

Esta familia de arquitecturas cuenta con 8 variantes (b0 a b7) en función de la potecia computacional y memoria requerida. En nuestro caso nos interesan especialmente la versión b3 y b4, pues son las primeras que proporcionan mejores resultados sobre el conjunto ImageNet que ResNet-50. 

Según la documentación de PyTorch, la versión b3 requiere 2.23 veces menos cálculos que ResNet-50, mientras que la versión b4 requiere aproximadamente el mismo número de cálculos. Por tanto, se decidió probar a emplear un clasificador basado en EfficientNet_b3. Sin embargo, a la hora de realizar el entrenamiento, la versión b3 necesita de unos 8 minutos por época para el conjunto de datos pequeño (teoricamente 10 veces más para el conjunto de datos completo). 

Dados los altos tiempos de entrenamiento, nos vemos a obligados a desistir y bajar a versiones "inferiores". Se realizaron pruebas con las versiones b2 y b1, pero finalmente tuvimos que usar la más básica, b0.

En esta tanda de experimentos se decidió utilizar una versión alternativa del optimizador ADAM, llamada ADAMW, que calcula la función de decaimiento de los pesos de forma distinta. También se cambia el "scheduler" para que la tasa de aprendizaje se reduzca de forma multiplicataiva cada 4 épocas, en lugar de forma exponencial como se venía realizando hasta el momento.

In [40]:
BEST_EFFICIENTNET_X20_PATH = os.path.join(os.path.join(os.getcwd(), 'results'), 'model_EfficientNet_Transfer_x20_ep=18_acc=0.9479166666666666.pt')
BEST_EFFICIENTNET_X200_PATH = os.path.join(os.path.join(os.getcwd(), 'results'), 'model_EfficientNet_Transfer_x200_ep=26_acc=0.7976973684210527.pt')

In [41]:
if TRAIN_EFFICIENTNET_MODELS and TRAIN_SMALL_MODELS:
    model_efficient_x20 = tv.models.efficientnet_b0(weights=tv.models.EfficientNet_B0_Weights.DEFAULT).to(DEVICE)
    criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.2)
    criterion.to(DEVICE)
    efficient_optimizer = torch.optim.AdamW(model_efficient_x20.parameters(), lr=1e-3)
    efficient_scheduler = torch.optim.lr_scheduler.StepLR(efficient_optimizer, step_size=4, gamma=0.96)
    if EXECUTING_FOR_FIRST_TIME:
        print(model_efficient_x20)
        torchsummary.summary(model_efficient_x20, (3, 320, 320))

In [42]:
if TRAIN_EFFICIENTNET_MODELS and TRAIN_SMALL_MODELS:
    train_loss_efficient_x20, train_acc_efficient_x20, val_loss_efficient_x20, val_acc_efficient_x20 = list(), list(), list(), list()

    EFFICIENTNET_X20_RESULTS = train_validate_model(model_efficient_x20, efficient_scheduler, efficient_optimizer, train_loader_x20, val_loader_x20, DATASET_20, 30, 'efficientnet', True)
    BEST_EFFICIENTNET_X20_PATH = EFFICIENTNET_X20_RESULTS[0]
    train_loss_efficient_x20, train_acc_efficient_x20, val_loss_efficient_x20, val_acc_efficient_x20 = EFFICIENTNET_X20_RESULTS[1:]
if TEST_SMALL_MODELS:
    efficient_x20_accuracy = test_model(model_efficient_x20, test_loader_x20, BEST_EFFICIENTNET_X20_PATH)

In [43]:
if TRAIN_EFFICIENTNET_MODELS and TRAIN_SMALL_MODELS and GENERATE_GRAPHS:
    graph_loss_function(train_loss_efficient_x20, val_loss_efficient_x20)
    graph_accuracy_function(train_acc_efficient_x20, val_acc_efficient_x20)

Para el mejor entrenamiento, con el ya habitual tamaño de lote 16, se ha conseguido una precisión sobre el conjunto pequeño del 88.3% tras 4 minutos y 57 segundos de entrenamiento. Estos resultados son un 5.7% peores que los obtenidos con ResNet-50. 

Respecto de la evolución de la función de pérdida y precisión podemos observar unas gráficas con valores alternos y oscilantes en un determinado rango para el conjunto de validación. En la mayoría de entrenamientos realizados con la arquitectura ResNet-50 estás gráficas eran mucho más suaves. Aparentemente seguimos sufriendo de "overfitting" en tanto que la pérdida para el conjunto de validación oscila, pero para el conjunto de entrenamiento tiende a descender. Dado que la precisión para validación fluctuaba decidimos no detener el entrenamiento de forma prematura; de hecho, los mejores resultados se consiguieron en la época 18, estando tentados de haberlo detenido cerca de la época 12.

![Loss efficientnet x20](./img_for_docs/efficientnet_b0_loss_x20.png)
![Acc efficientnet x200](./img_for_docs/efficientnet_b0_acc_x20.png)

In [44]:
if TRAIN_EFFICIENTNET_MODELS and TRAIN_BIG_MODELS:
    model_efficient_x200 = tv.models.efficientnet_b0(weights=tv.models.EfficientNet_B0_Weights.IMAGENET1K_V1).to(DEVICE)
    criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.2)
    criterion.to(DEVICE)
    efficient_optimizer = torch.optim.AdamW(model_efficient_x200.parameters(), lr=1e-4)
    efficient_scheduler = torch.optim.lr_scheduler.StepLR(efficient_optimizer, step_size=4, gamma=0.96)

    train_loss_efficient_x200, train_acc_efficient_x200, val_loss_efficient_x200, val_acc_efficient_x200 = list(), list(), list(), list()

    EFFICIENTNET_X200_RESULTS = train_validate_model(model_efficient_x200, efficient_scheduler, efficient_optimizer, train_loader_x200, val_loader_x200, DATASET_200, 30, 'efficientnet', True)
    BEST_EFFICIENTNET_X200_PATH = EFFICIENTNET_X200_RESULTS[0]
    train_loss_efficient_x200, train_acc_efficient_x200, val_loss_efficient_x200, val_acc_efficient_x200 = EFFICIENTNET_X200_RESULTS[1:]
if TEST_BIG_MODELS:
    efficient_x200_accuracy = test_model(model_efficient_x200, test_loader_x200, BEST_EFFICIENTNET_X200_PATH)

In [45]:
if TRAIN_EFFICIENTNET_MODELS and TRAIN_BIG_MODELS and GENERATE_GRAPHS:
    graph_loss_function(train_loss_efficient_x200, val_loss_efficient_x200)
    graph_accuracy_function(train_acc_efficient_x200, val_acc_efficient_x200)

Para el mejor entrenamiento sobre el conjunto completo se ha obtenido una precisión del 78.4% tras 43 minutos y 10 segundos de entrenamiento. Estos resultados son un 3.7% peores que los obtenidos con ResNet-50. 

En esta ocasión, si nos fijamos en las gráficas adjuntas a continuación, podemos observar una tendencia mucho más suave, en línea con lo obtenido en experimentos con otros modelos. Volvemos a observar "overfitting" dada la continua disminución de la función de pérdida para el conjunto de entrenamiento mientras que se estanca para el conjunto de validación. Para la precisión, el modelo está cerca obtener precisiones perfectas para el conjunto de entrenamiento, mientras que para el conjunto de validación oscila entre el 74 y 78%.

![Loss efficientnet x20](./img_for_docs/efficientnet_b0_loss_x200.png)
![Acc efficientnet x200](./img_for_docs/efficientnet_b0_acc_x200.png)

## Clasificador MobileNetV3

En los apartados anteriores conseguimos obtener clasificadores que funciona nrazonablemente bien, pero cuyo tiempo de entrenamiento y ejecución puede ser alto en función de si se entrena desde cero o no y el número de épocas empleadas.

El disponer de un modelo de clasificación para aves puede resultar especialmente útil para investigadores o aficionados que deseen conocer más datos sobre la especie de pájaro que han observado en su último viaje. Sin embargo, no todo el mundo dispone de equipos pórtatiles, o incluso de sobremesa, con GPU y los requisitos para ejecutarlos. Si pensamos en el aficionado o investigador que va al monte y ha fotografiado un pájaro desconocido, es lógico pensar que querría usar su teléfono móvil para obtener más información sobre dicho pájaro.

La aplicación con la que realizar esta consulta podría mandar la foto a un servidor que ejecute el clasificador y devolver el resultado, pero no siempre disponemos de cobertura a la red de datos móviles en entornos rurales, por lo que convendría realizar estos cálculos localmente en el dispostivo. Es en estas situaciones donde convendría emplear modelos menos costosos en tiempo de procesamiento y memoria, aunque se realice un pequeño sacrificio en la precisión de los resultados. Así pues, entran al tablero de juego modelos ligeros como MobileNetV3.

MobileNetV3 es una arquitectura de red neuronal convolucional (CNN) diseñada específicamente para su implementación en dispositivos móviles y aplicaciones de visión por computadora en tiempo real. Fue propuesto por investigadores de Google en su artículo "Searching for MobileNetV3" en 2019. El objetivo principal de este modelo es lograr un equilibrio entre la precisión y la eficiencia computacional, es decir, obtener modelos de alta calidad que sean lo más livianos y rápidos posible.

Dados los pésimos resultados que obtuvimos al intentar entrenar ResNet-50 desde cero, en este apartado abordaremos únicamente el proceso de transferencia de aprendizaje.

In [46]:
model_mobilenet_x20 = tv.models.mobilenet_v3_large(weights=tv.models.MobileNet_V3_Large_Weights.DEFAULT).to(DEVICE)
mobilenet_optimizer = torch.optim.Adam(model_mobilenet_x20.parameters(), lr=1e-3)
mobilenet_scheduler = torch.optim.lr_scheduler.ExponentialLR(mobilenet_optimizer, gamma=0.95)

if EXECUTING_FOR_FIRST_TIME:
    print(model_mobilenet_x20)
    torchsummary.summary(model_mobilenet_x20, (3, 224, 224))

La creación del modelo y su optimizador y controlador de la tasa de aprendizaje asociado es análoga a la empleada anteriormente para ResNet, simplemente debemos cambiar la clase a instanciar. PyTorch dispone de dos variantes de la red, en función de los requisitos computacionales y de memoria. Para nuestros experimentos decidimos utilizar la mayor de las dos, pues a pesar de tener el triple coste computacional obtiene un 9% más de precisión sobre el conjunto de datos de prueba de ImageNet. Esta versión tiene 5.483.032 parámetros entrenables (unas 4.3 veces menos que ResNet-50) y requiere de 126.90 MB (unas 7.15 veces menos que ResNet-50)

Respecto del modelo ResNet-50 que hemos empleado anteriormente, la documentación de PyTorch indica que MobileNetV3-Large requiere de 18.5 veces menos operaciones por segundo aproximadamente. No obtendremos una ganancia proporcional en tiempo de ejecución, pero si que hemos observado una reducción en el tiempo de ejecución cercana a un factor 2.25.  

In [47]:
BEST_MOBILENET_X20_PATH = os.path.join(os.path.join(os.getcwd(), 'results'), 'model_MobileNet_Transfer_x20_ep=10_acc=0.8697916666666666.pt')  
BEST_MOBILENET_X200_PATH = os.path.join(os.path.join(os.getcwd(), 'results'), 'model_MobileNet_Transfer_x200_ep=30_acc=0.7582236842105263.pt') 

In [48]:
train_loss_mobilenet_x20, train_acc_mobilenet_x20, val_loss_mobilenet_x20, val_acc_mobilenet_x20 = list(), list(), list(), list()

if TRAIN_MOBILENET_MODELS and TRAIN_SMALL_MODELS:
  MOBILENET_X20_RESULTS = train_validate_model(model_mobilenet_x20, mobilenet_scheduler, mobilenet_optimizer, train_loader_x20, val_loader_x20, DATASET_20, 40, 'mobilenet', True)
  BEST_MOBILENET_X20_PATH = MOBILENET_X20_RESULTS[0]
  train_loss_mobilenet_x20, train_acc_mobilenet_x20, val_loss_mobilenet_x20, val_acc_mobilenet_x20 = MOBILENET_X20_RESULTS[1:]
if TEST_SMALL_MODELS:
  mobilenet_x20_accuracy = test_model(model_mobilenet_x20, test_loader_x20, BEST_MOBILENET_X20_PATH)

In [49]:
if TRAIN_MOBILENET_MODELS and TRAIN_SMALL_MODELS and GENERATE_GRAPHS:
    graph_loss_function(train_loss_mobilenet_x20, val_loss_mobilenet_x20)
    graph_accuracy_function(train_acc_mobilenet_x20, val_acc_mobilenet_x20)

Dado que la red es en principio más sencilla y está pensada para usarse en dispositivos móviles en tiempo real, los resultados son algo peores que los obtenidos con RESNET50 en el apartado anterior.

El mejor entrenamiento realizado permite generar un modelo con el que clasificar a las aves del conjunto de prueba pequeño correctamente un 89.5%. El entrenamiento en cuestión tenía como tamaño de lote 16 imágenes, se ha llevado a cabo durante 40 épocas y ha tardado en ejecutarse 4 minutos y 9 segundos. Distintas combinaciones de entrenamientos con tamaño de lote 24, más épocas y distintos parametros LR y gamma del optimizador y "scheduler" proporcionan resultados relativamente similares, siendo la menor de las precisiones obtenidas del 85.7%.

Nótese que la diferencia de precision respecto del modelo RESNET50 para el conjunto pequeño es de apenas un 4%, dentro de un rango de precisiones muy altas. Por su parte, el tiempo de entrenamiento ha sido menor incluso ejecutando un mayor número de épocas.

En lo que respecta a las curvas de la función de pérdida y precisión, podemos ver en las siguientes gráficas como salvo para el caso de la función de pérdida en validación no se ha experimentado un cambio significativo durante el transcurso del entrenamiento. Como en ocasiones anteriores, podemos sospechar de sufrir "overfitting" al tenerse precisiones en entrenamiento cercanas a 1 desde épocas tempranas. 

![Mobilenet loss x20](./img_for_docs/mobilenet_loss_x20.png)
![Mobilenet acc x20](./img_for_docs/mobilenet_acc_x20.png)

Sobre el conjunto de validación no se obtuvieron mejoras en la precisión desde la época 10, por lo que se podría haber detenido tempranamente el entrenamiento y haber empleado solamente unos 60 segundos en realizarlo, sin presumiblemente existir mucha diferencia en los resultados finales.

No en todas las ejecuciones se ha observado este ajuste tan rápido al conjunto de entrenamiento y validación; en alguna de ellas se ha comenzado con una precisión sobre el conjunto de validación cercana al 40%, que fue mejorando y alcanzó niveles cercanos a los mostrados en torno a la época 12.

In [50]:
model_mobilenet_x200 = tv.models.mobilenet_v3_large(weights=tv.models.MobileNet_V3_Large_Weights.DEFAULT).to(DEVICE)
mobilenet_optimizer = torch.optim.Adam(model_mobilenet_x200.parameters(), lr=1e-4)
mobilenet_scheduler = torch.optim.lr_scheduler.ExponentialLR(mobilenet_optimizer, gamma=0.90)

train_loss_mobilenet_x200, train_acc_mobilenet_x200, val_loss_mobilenet_x200, val_acc_mobilenet_x200 = list(), list(), list(), list()

if TRAIN_MOBILENET_MODELS and TRAIN_SMALL_MODELS:
  MOBILENET_X200_RESULTS = train_validate_model(model_mobilenet_x200, mobilenet_scheduler, mobilenet_optimizer, train_loader_x200, val_loader_x200, DATASET_200, 30, 'mobilenet', True)
  BEST_MOBILENET_X200_PATH = MOBILENET_X200_RESULTS[0]
  train_loss_mobilenet_x200, train_acc_mobilenet_x200, val_loss_mobilenet_x200, val_acc_mobilenet_x200 = MOBILENET_X200_RESULTS[1:]
if TEST_SMALL_MODELS:
  mobilenet_x200_accuracy = test_model(model_mobilenet_x200, test_loader_x200, BEST_MOBILENET_X200_PATH)

In [51]:
if TRAIN_MOBILENET_MODELS and TRAIN_BIG_MODELS and GENERATE_GRAPHS:
    graph_loss_function(train_loss_mobilenet_x200, val_loss_mobilenet_x200)
    graph_accuracy_function(train_acc_mobilenet_x200, val_acc_mobilenet_x200)

Tras 27 minutos y 55 segundos obtenemos los mejores resultados para un entrenamiento del modelo MobileNetV3 sobre el conjunto de datos completo. En este caso, la precisión desciende hasta el 74.8%, siendo esta un 7.3% menor que la obtenida por el procedimiento equivalente que hace uso de ResNet-50. A cambio, podemos destacar que el tiempo de ejecución fue unas 2.5 veces menor. 

Si observamos la evolución de la función de pérdida podemos intuir la existencia de "overfitting" a partir de la sexta o séptima época dado que la pérdida para el conjunto de entrenamiento sigue reduciendose mientras que la de validación comienza a oscilar en un rango pequeño acotado. 

En la gráfica de la precisión podemos ver como en esta ocasión el modelo ha tardado varias épocas en alcanzar una precisión aceptable y dejar de mejorarla. En cualquier caso, se produce un ajuste casi perfecto al conjunto de entrenamiento una vez se alcanzan las 15 épocas. La mejor precisión sobre el conjunto de validación se produce justamente en la última época, mejorando en 0.66% el resultado anterior. Dudamos seriamente que emplear un mayor número de épocas hubiese acabado proporcionando mejores resultados, pues en ejecuciones de 40 y 50 épocas se obtuvieron resultados ligeramente peores y en esta ejecución concreta se llevaba manteniendo la tendencia durante un número considerable de épocas.

El comportamiento expuesto en estas gráficas recuerda al explicado en el apartado anterior para el modelo ResNet-50.

![Mobilenet loss x200](./img_for_docs/mobilenet_loss_x200.png)
![Mobilenet acc x200](./img_for_docs/mobilenet_acc_x200.png)

## Gráfica comparativa entre modelos

In [52]:
# Values are hardcoded as multiple test have been executed for each model, we take the best from each
if GENERATE_GRAPHS:
  logs = {}
  logs['Setups'] = ['Baseline ResNet', 'Transfer ResNet', 'Transfer MobileNetV3', 'Transfer EfficientNet_B0']
  logs['Accuracy'] = [0.47, 0.9375, 0.895, 0.883]
  logs['Time'] = [16.5, 6.85, 4.33, 4.95] # Extrapolated to decimal base

  cmap = mpl.colormaps['Blues_r']
  colors = [cmap(0.9), cmap(0.5), cmap(0.1)]
  x_min, x_max = np.min(logs['Accuracy']), np.max(logs['Accuracy'])

  plt.barh(logs['Setups'], logs['Accuracy'], color=colors)
  plt.xlim(x_min - 5e-2, x_max + 5e-2)
  plt.xlabel('Accuracy (%)')
  plt.ylabel('Setups')
  plt.show()

  plt.barh(logs['Setups'], logs['Time'], color=colors)
  plt.xlim(3.8, 17)
  plt.xlabel('Execution Time (minutes)')
  plt.ylabel('Setups')
  plt.show()

  logs2 = {}
  logs2['Setups'] = ['Baseline ResNet', 'Transfer ResNet', 'Transfer MobileNetV3', 'Transfer EfficientNet_B0']
  logs2['Accuracy'] = [0.2335, 0.821, 0.748, 0.784]
  logs2['Time'] = [40.37, 64.66, 27.92, 44.17] # Extrapolated to decimal base

  cmap = mpl.colormaps['Blues_r']
  colors = [cmap(0.9), cmap(0.5), cmap(0.1)]
  x_min, x_max = np.min(logs2['Accuracy']), np.max(logs2['Accuracy'])

  plt.barh(logs2['Setups'], logs2['Accuracy'], color=colors)
  plt.xlim(x_min - 5e-2, x_max + 5e-2)
  plt.xlabel('Accuracy (%)')
  plt.ylabel('Setups')
  plt.show()

  plt.barh(logs2['Setups'], logs2['Time'], color=colors)
  plt.xlim(25, 67)
  plt.xlabel('Execution Time (minutes)')
  plt.ylabel('Setups')
  plt.show()

En las siguientes gráficas se puede observar una comparativa directa entre la precisión obtenida para cada modelo y el tiempo de entrenamiento que ha requerido para los conjuntos de datos con 20 clases (izquierda) y 200 clases (derecha). Téngase en cuenta que todos los entrenamientos constan de 30 épocas, a excepción de "Baseline ResNet" para el conjunto de datos pequeño, que requirió de 70 épocas ya que nos dimos cuenta que la mejora continuaba más allá de las 30 épocas y esta era significativa; y el entrenamiento de MobileNet para el conjunto pequeño, que consta de 40 épocas.

![Acc x20 comparativa](./img_for_docs/comparativa_acc_x20.png)
![Acc x200 comparativa](./img_for_docs/comparativa_acc_x200.png)
![Training time x20 comparativa](./img_for_docs/execution_time_x20.png)
![Training time x200 comparativa](./img_for_docs/execution_time_x200.png)

Obviando los desastrosos resultados obtenidos al intentar entrenar un clasificador desde cero, podemos ver como ResNet-50 proporciona buenas precisiones para ambos conjuntos tras ser entrenado en un tiempo relativamente razonable. Así también, podemos ver como MobileNetV3 ofrece resultados algo peores, pero que están dentro de los márgenes de lo que podríamos considerar aceptable, todo ello requeriendo un tiempo de entrenamiento significativamente menor. Cabe también destacar que los tiempos de inferenica de MobileNetV3 son menores que los empleados por ResNet-50. 

En lo relativo a EfficientNet_b0, esta obtiene un peor desempeño sobre el conjunto de datos pequeño que el resto de modelos; pero para el conjunto completo consigue superar ligeramente a MobileNetV3. Nótese que el tiempo de ejecución necesario para su entrenamiento se encuentra entre los propios de los otros modelos. Según la documentación de PyTorch necesita del triple de operaciones que MobileNetV3, por lo que podría no ser apto para dispositivos móviles. Sin embargo, si que podría ser útil fuera de este ámbito, ya que teóricamente requiere de 5 veces menos operaciones que ResNet-50 y ofrece resultados bastante razonables con tiempos de entrenamiento entre 2 y 3 veces menores.

Todos los modelos obtienen sobre nuestro conjunto de datos precisiones algo mayores que las obtenidas en sus entrenamientos originales sobre ImageNet (de acuerdo a la documentación de PyTorch).

Como se mencionó en apartados anteriores, se obvia EfficientNet_b3 y b4 de la comparativa al no haber podido terminar los entrenamientos correspondientes. De seguir la tendencia de mejora que hemos venido observando entre los pesos originales sobre ImageNet y los nuestros sobre CUB 200-2011, estimamos que podríamos obtener entre un 2% y 4% de mejora en la precisión de las predicciones sobre el conjunto de datos con 200 clases. Dado el elevadísimo tiempo de ejecución que sería necesario (estimamos varias horas para el conjunto completo a partir de lo que ha tardado para una época del conjunto pequeño); decidimos que no merece la pena un sacrificio en tiempo tan grande por una ligera mejora en precisión, máxime una vez hemos aceptado niveles aceptables con ResNet-50 y algo menores con la variante más "básica" de la familia EfficientNet.

## Conclusión

En esta práctica se ha abordado el problema de la creación de un modelo capaz de clasificar los pájaros del conjunto de datos CUB 200-2011, el cual presenta 200 clases distintas de pájaros y pocas muestras para cada clase (unas 30), lo que dificulta la tarea.

Tras la realización de la práctica, y en virtud de los experimentos y resultados obtenidos, podemos concluir lo siguiente:

- Dado el escaso número de muestras por clase resulta inviable crear un modelo desde cero con el que conseguir una buena generalización y predicciones acertadas, a pesar de emplear técnicas de aumento de datos. La precisión para el conjunto de datos pequeño haciendo uso de ResNet-50 apenas llega al 47%, mientras que para el conjunto de datos completo no se ha conseguido obtener una precisión superior al 23.5%.
- A la luz de los resultados expuestos en el punto anterior, nos vemos obligados a partir de pesos pre-entrenados para conjuntos de datos mucho más grandes, como ImageNet, y realizar un proceso de transferencia de aprendizaje con el que dotar al modelo de cierto conocimiento sobre nuestro conjunto de datos y problemática concreta.
- El uso de transferencia de aprendizaje permite reducir considerablemente el tiempo de entrenamiento y permite obtener unos resultados mucho mejores. Para la arquitectura ResNet-50 conseguimos una precisión del 94% sobre el conjunto de datos pequeño y de 82.1% sobre el conjunto de datos completo.
- Resulta esencial elegir una arquitectura base de acuerdo a ciertos criterios predefinidos. En nuestro caso, se eligió ResNet-50 por ser la que mejor relación entre coste computacional y precisión parecía proporcionar de entre aquellas disponibles en PyTorch que pudimos llegar a ejecutar. Somos conscientes de que con arquitecturas más complejas como EfficientNet (a partir de la versión B3) o incluso EfficientNetV2 podríamos obtener mejores resultados (estimamos que entre un 2 y 4% más de precisión sobre el conjunto de datos completo si usaramos la versión B4), pero hemos sido incapaces de completar entrenamientos con estas versiones dados los altos tiempos de ejecución requeridos. La versión básica de EfficientNet proporciona peores resultados que ResNet-50, pero con tiempos de entrenamiento menores.
- En el ámbito de arquitecturas aptas para dispositivos móviles y tiempo real, decidimos probar a ajustar un modelo MobileNetV3 entrenado sobre el conjunto de datos de ImageNet. Los resultados obtenidos son ligeramente peores que los proporcionados por ResNet-50 (4% para el conjunto de datos pequeño y 7.4% para el conjunto completo); pero el tiempo de entrenamiento es significativamente menor, ídem para el tiempo de inferencia.

## Bibliografía

Durante la realización de esta práctica se ha consultado las siguientes fuentes de información:
- [Bird by Bird using Deep Learning](https://towardsdatascience.com/bird-by-bird-using-deep-learning-4c0fa81365d7)
- [Bird by Bird using Deep Learning repository](https://github.com/slipnitskaya/caltech-birds-advanced-classification)
- [Image Classification of birds](https://github.com/ecm200/caltech_birds)
- [pytorch-cubbirds200-classification](https://www.kaggle.com/code/sharansmenon/pytorch-cubbirds200-classification/notebook)
- [A Visual Guide to Learning Rate Schedulers in PyTorch](https://towardsdatascience.com/a-visual-guide-to-learning-rate-schedulers-in-pytorch-24bbb262c863)
- [Image classification: ResNet vs EfficientNet vs EfficientNet_v2 vs Compact Convolutional Transformers](https://medium.com/@enrico.randellini/image-classification-resnet-vs-efficientnet-vs-efficientnet-v2-vs-compact-convolutional-c205838bbf49)
- [Documentación de PyTorch](https://pytorch.org/vision/main/index.html)
    - [MODELS AND PRE-TRAINED WEIGHTS](https://pytorch.org/vision/main/models.html)
        - [RESNET50](https://pytorch.org/vision/main/models/generated/torchvision.models.resnet50.html#torchvision.models.resnet50)
        - [MOBILENET_v3_LARGE](https://pytorch.org/vision/main/models/generated/torchvision.models.mobilenet_v3_large.html#torchvision.models.mobilenet_v3_large)
        - [EFFICIENTNET](https://pytorch.org/vision/main/models/efficientnet.html)
    - [FINETUNING TORCHVISION MODELS](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html)
- [Documentación de Matplotlib](https://matplotlib.org/stable/index.html)
- [Documentación de Scikit-Learn](https://scikit-learn.org/stable/modules/classes.html#)
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)
- [Searching for MobileNetV3](https://arxiv.org/abs/1905.02244)
- Apuntes de la asignatura "Sistemas Inteligentes para la Gestión en la Empresa" del Máster en Ingeniería Informática de la Universidad de Granada
- Apuntes de la asignatura "Inteligencia Computacional" del Máster en Ingeniería Informática de la Universidad de Granada